In [1]:
import pandas
import graphlab

In [2]:
review_df = pandas.read_csv('../output/review_data_frame2.csv')

In [3]:
review_df = review_df[['product_productid', 'review_text', 'review_score']]

In [4]:
review_df = graphlab.SFrame(review_df)

This non-commercial license of GraphLab Create is assigned to voelunteer@gmail.com and will expire on December 23, 2016. For commercial licensing options, visit https://dato.com/buy/.


2016-04-09 15:22:10,447 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1460229729.log


In [6]:
total_review_df = review_df

In [5]:
review_df = review_df['review_text', 'review_score']

In [7]:
review_df = review_df.unique()

In [8]:
# sentiment_model = graphlab.product_sentiment.create(review_df, features=['review_text'])

In [9]:
# review_df = sentiment_model.get_most_negative(k = 134881)

In [8]:
review_df 

review_score,review_text
4,As a major fan of RobertRodriguez and a fan of ...
5,When I compare this HBOminiseries to other ...
4,This is a review of theDisney Blu-Ray releas ...
1,"The characters lackdepth, background, ..."
5,"""Tangled"" is spectacular,but it's not because ..."
5,I love watching JeffDunham do his thing. ...
1,Though M. NightShyamalan's unable to ...
5,5.0 stars<br /><br />Anexemplary film and ea ...
1,"I had very, very highhopes for this film - ..."
2,"No, you don't have to seeall my reviews. I try to ..."


In [9]:
review_df['word_count'] = graphlab.text_analytics.count_words(review_df['review_text'])

In [10]:
review_df['tf-idf'] = graphlab.text_analytics.tf_idf(review_df['word_count'])

In [11]:
# k_means_model = graphlab.kmeans.create(review_df, num_clusters = 5, max_iterations=30, features=['tf-idf'])

In [12]:
# k_means_model.save('../output/K(5) Means Model (new)')

In [13]:
k_means_model = graphlab.load_model('../output/K(5) Means Model')

In [14]:
k_means_model.get('cluster_id')

row_id,cluster_id,distance
0,0,41.9870109558
1,0,37.276184082
2,0,31.915927887
3,0,31.7485027313
4,0,63.2005653381
5,0,30.2194385529
6,1,107.159637451
7,0,55.3383369446
8,0,37.1716957092
9,0,50.0964927673


In [15]:
review_df['cluster_id'] = k_means_model.get('cluster_id')['cluster_id']

In [16]:
review_df

review_score,review_text,word_count,tf-idf,cluster_id
4,As a major fan of RobertRodriguez and a fan of ...,"{'all': 1,'particularly': 1, ...","{'all':0.8239219583239975, ...",0
5,When I compare this HBOminiseries to other ...,"{'and': 1, 'this': 3,'brothers,': 1, 'on': 1, ...","{'and':0.05758588282085063, ...",0
4,This is a review of theDisney Blu-Ray releas ...,"{'and': 5, 'available':1, 'pictures,': 1, ...","{'and':0.28792941410425316, ...",0
1,"The characters lackdepth, background, ...","{'atmosphere': 2, 'just':2, 'lack': 1, 'movies.': ...","{'atmosphere':9.38321110884002, 'ju ...",0
5,"""Tangled"" is spectacular,but it's not because ...","{'thomas': 1,'selection': 1, 'gold': ...","{'thomas':4.94613541955489, ...",0
5,I love watching JeffDunham do his thing. ...,"{'jeff': 1, 'and': 4,'there': 4, 'love': 1, ...","{'jeff':3.770830510098526, 'a ...",0
1,Though M. NightShyamalan's unable to ...,"{'all': 1, ""don't"": 1,'being': 1, 'awful': 1, ...","{'all':0.8239219583239975, ...",1
5,5.0 stars<br /><br />Anexemplary film and ea ...,"{'just': 1, ""don't"": 1,'patina': 1, 'timeles ...","{'just':0.9724312321670323, ...",0
1,"I had very, very highhopes for this film - ...","{'all': 2,'furthermore,': 1, ...","{'all':1.647843916647995, ...",0
2,"No, you don't have to seeall my reviews. I try to ...","{'no,': 1, 'all': 1,'reviews.': 1, 'just' ...","{'no,': 4.57008419665302,'all': ...",0


In [18]:
review_df_new = review_df['review_text', 'review_score', 'cluster_id']

In [19]:
review_df_new.save('../output/cluster_of_review.csv')

In [21]:
review_df['cluster_id'].show()

Canvas is accessible via web browser at the URL: http://localhost:51842/index.html
Opening Canvas in default web browser.


In [33]:
cluster_topics = []

In [34]:
for cluster_id in range(0, 5):
    cluster_doc = review_df[review_df['cluster_id'] == cluster_id]['tf-idf']
    cluster_topic_model = graphlab.topic_model.create(cluster_doc)
    cluster_topics.append(cluster_topic_model.get_topics())

Learning a topic model

Number of documents     17250

Vocabulary size    154375

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 13.072ms      | 110333         | 0               |

+-----------+---------------+----------------+-----------------+

In [50]:
cluster_topics[4].save('../output/cluster_4_topics.csv')

dtype: list
Rows: 5
[[{'topic': 0, 'score': 0.00292665907406073, 'word': '/><br'}, {'topic': 0, 'score': 0.0020120052968477867, 'word': 'they'}, {'topic': 0, 'score': 0.0019368197621719944, 'word': 'he'}, {'topic': 0, 'score': 0.0019334022378685494, 'word': 'his'}, {'topic': 0, 'score': 0.0018936346823375516, 'word': 'as'}, {'topic': 1, 'score': 0.0025433582501408684, 'word': 'her'}, {'topic': 1, 'score': 0.0024123145537315494, 'word': '/>'}, {'topic': 1, 'score': 0.0019303402804972745, 'word': '/><br'}, {'topic': 1, 'score': 0.001905908404895537, 'word': 'jack'}, {'topic': 1, 'score': 0.0018370549372906406, 'word': 'she'}, {'topic': 2, 'score': 0.002234115568014174, 'word': 'remy'}, {'topic': 2, 'score': 0.0020936051549315213, 'word': '/>'}, {'topic': 2, 'score': 0.0020431655194659538, 'word': 'harry'}, {'topic': 2, 'score': 0.0016216342802179964, 'word': 'potter'}, {'topic': 2, 'score': 0.001499138022658761, 'word': 'evolution'}, {'topic': 3, 'score': 0.002635957067620968, 'word': 'd